[toc]

# XGBoost & GBDT 面试题

## 1. XGBoost和GBDT有什么不同

1. 基分类器： 
    - >XGBoost 支持 CART树，也支持线性分类器，此时 XGBoost 相当于带 L_1 和 L_2 正则化的 Linear Regression(for regression) 或 Logistic Regression(for classification) 
    - 这段是网上抄的，但是感觉说的不是很有道理。实际上，个人认为它们的基学习器最大的区别应该是目标函数的不同。GBDT 的 基学习器使用的是回归树。回归树的损失函数是 MSE。而 XGBoost 使用是的修改后的回归树，损失函数是对整体 loss 二阶 Taylor 展开的结果。

2. 导数信息
    - XGBoost用到了二阶信息
    - GBDT只用到了一阶信息

3. 正则项
    - XGBoost使用了正则项。由两部分组成，一部分是对叶结点数量做出的惩罚，另一部分是对每个叶结点上的输出分数的惩罚。
    - GBDT 没有使用正则项

4. 列采样
    - XGBoost使用了类似于 RandomForest 的列采样以减少过拟合
    - GBDT没有使用

5. 缺失值处理
    - XGBoost有缺失值处理。会尝试将缺失值分到左右的结点上去并选择使用损失减少最多的那个作为缺失值的分类。
    - GBDT 没有缺失值处理

6. 并行化
    - XGBoost可以在**特征级别**上进行并行化
    - GBDT不行。（其实个人认为 GBDT 也可以在特征级别上进行并行化，只需要借鉴 XGBoost 的那部分就可以了）

## 2. XGBoost为什么可以并行


XGBoost的并行不是每棵树进行并行，它本质上还是 Boosting 方法，所以不能完成每个基分类器的并行训练。

XGBoost的并行指的是**特征粒度的并行**，不是tree粒度的并行。XGBoost在选取最佳分割点的时候不同的特征可以分别计算，最后选择最优的那个特征来进行分割。

## 3. XGBoost为什么快

因为XGBoost做了许多优化以适应大规模数据集的训练。

1. 分桶：XGBoost在选择划分点时使用的是贪心算法。对于比较大规模的数据集来说，XGBoost可以使用Approximate Greedy Algorithm 来实行计算。通过分桶的方式的方式来样大数据的颗粒度，以加速计算。

2. block 中的数据只需要进行一次排序，之后都使用之前的排序结果，因此可以减少排序。

3. 列采样。XGBoost中使用了 Random Forest 中的列采样，在防止过拟合的同时，也可以减少计算量（因为可以少算一些特征）

4. 对特征粒度进行并行。 

5. cache aware accesss。XGBoost在为了避免cache miss使用了数据预取的计算。对于 Approximate Greedy Algorithm 来说，还可以调整合适的 blocksize 以减少 cache miss

## 4. XGBoost如何防止过拟合

1. 目标函数添加正则项。正则项分为两个部分，一个是对叶结点数量的惩罚，一个是对每个叶结点上输出分数的惩罚。
2. 列采样，即可以防止过拟合，还可以减少需要计算分裂点的特征数量。
3. shrinkage，采用下面的式子来更新

$$
\hat{y}^{(t)}  = \hat{y}^{(t-1)} + \eta f_t
$$ 

## 5. XGBoost的推导

## 6. Todo - 2020-04-25 16:24 -- by ed XGBoost如何处理不平衡样本

## 7. XGBoost中如何对树进行剪枝

1. 添加正则项本身就相当于预剪枝。

## 8. XGBoost如何选择最佳分割点？

1. 首先，选择最佳分割点的策略是增益最大

$$
L_{split} = \frac{1}{2}[\frac{G_L}{H_L+\lambda} + \frac{G_R}{H_R+\lambda}  - \frac{G}{H+\lambda}]  - \gamma
$$ 

2. 分割特征的选取采用了类型于 RandomForest 中的列采样计算，随机选取一些特征作为候选特征，而非使用全部特征。

3. 具体的选取算法有 Exact Greedy Algorithm 和 Approximate Greedy Algorithm。而 Approximate Greedy Algorithm 需要对样本进行分桶。分桶是通过加权二阶导数来进行的。Approximate Greedy Algorithm 还有 global 和 local 划分的区别。

global 是指在最开始给出候选分割点之后，接下来都使用这些候选分割点去计算。而 local 是最每次分裂都先计算出新的候选分割点，再在这堆新的候选分割点中选择最好的那个。可以看到，local 的计算量更大一些，效果也相对比较好一些。

## 9. 为什么XGBoost相比某些模型对缺失值不敏感

实际上这个是所有树模型都具有的一个特性。

因为树模型在结点划分的时候是寻找最优的分裂点，这个分裂点的寻找不怎么会受到缺失值的影响，因此树模型对缺失值不敏感。

而且，在这个基础上，XGBoost还有专门处理缺失值的方法。

## Todo - 2020-04-25 17:55 -- by ed 10. 为什么XGBoost不使用处理高维稀疏问题？而 LR 更加适合？

这个问题实际上也是树模型的一个特性。

树模型不使用处理高维稀疏问题的原因有两个：
1. 分割时需要考虑所有的特征。因此特征越多，计算开销越大。
2. 其次，在高维稀疏问题上更容易过拟合。具体可以看 [2] 中的例子。

LR的目标就是找到一个超平面对样本是的正负样本位于两侧，由于这个模型够简单，不会出现gbdt上过拟合的问题，不过它可能要但心欠拟合的问题了。

讲真从这个角度来看，XGBoost算是树模型中处理高维稀疏问题比较好的了。一方面有列采样减少特征分裂中的时间。另一方面还对每个叶结点的输出分数有惩罚。



## 10. 为什么 RandomForest 的树比 GBDT / XGBoost 的要深一些？

因为 RandomForest 是 Bagging。Bagging 方差的目的是减少方差，但是不能减少偏差。因此 RandomForest 的基学习器需要有比较好的拟合能力才能保证 ensemble 之后的结果的 bias 比较小。因此， RandomForest 的树要深一些才行。

而 GBDT 是 Boosting 算法。Boosting 算法的目的是减小偏差，不能减小方差。为了保证方差足够小， Boosting 的树的深度可以适当浅一些，比较浅的树的拟合能力虽然弱，但是相对来说比较稳定，方差比较小（举个极端一点的例子，只有一片叶子的树，虽然预测能力很差，但是方差为0）

## 11. bagging和boosting是什么，有什么区别，是怎么把弱模型变成强模型的?

bagging 和 boosting 都是集成学习。集成学习的目的是将多个学习器的结果结合起来。其中 boosting 是串行结合，而 bagging 是并行结合。

### Boosting

boosting 的多个学习器是串行的，不同的学习器的训练过程是耦合的，只有学习完上一个学习器时，才能学习下一个学习器。而不同的 boosting 方法有不同的结合方式，如下：

- Adaboost 是通过改变样本的权重来实现的。Adaboost 会对上一次预测错误的结果施以较大的权重，训练下一个分类器的时候重点对之前分类错误的样本进行学习。
- Graident Boost 是通过学习之前的学习器的残差/负梯度来进行不同学习器的结合的。
- XGBoost 是通过最小化之前学习器学习完的损失来进行不同学习器的结合的。

###  Bagging

Bagging 是通过并行多个学习器来进行学习的，不同的学习器之前的学习是解耦的。Bagging 的全称是 Bootstrap Aggregating，其中，Bootstrap 是一种有放回的抽样方法。

结合的方法主要是 majority voting 和 averaging。

### 区别

1. bagging 可以并行，boosting 不能并行。
2. bagging 减少 variance，boosting 减少 bias。
3. bagging 的基学习器应该选择那种 bias 小，variance 大的基学习器，如不进行剪枝的决策树；而 boosting 的基学习器应该选择那种 bias 比较大的，但 variance 比较小的，如深度较浅的学习器。

## 12. Bagging 和 Boosting 的数学原理 / 如何从减小方差和偏差的角度解释 Boosting 和 Bagging 的原理 / 为什么说 Bagging 减小方差而 Boosting 减小偏差？

假设有 n 个随机变量 $x_i$，均服从均值为 $\mu$ 方差均为 $\sigma^2$ 的分布，且而两两之间的相关系数为 $\rho$，则 n 个随机变量的求平均的均值为

$$
\mathbb{E} \left( \sum_{i=1}^n\frac{x_i}{n}\right) = \mu
$$

则 $\rho = \frac{cov(x_i, x_j)}{\sigma^2}$

即
$$
cov(x_i, x_j) = \rho \sigma^2
$$

而其方差为

$$
\begin{aligned}
var\left( \sum_{i=1}^n\frac{x_i}{n}\right) &= \frac{1}{n^2} var \left( \sum_{i=1}^n x_i \right) \\
&= \frac{1}{n^2} \left( n\sigma^2 + n(n-1) \rho \sigma^2 \right)\\
&= \frac{\sigma^2}{n} + \frac{n-1}{n} \rho \sigma^2 \\
&= \frac{\sigma^2}{n} + \rho \sigma^2 - \frac{1}{n} \rho \sigma^2 \\
&= \rho \sigma^2 + \frac{1-\rho}{n} \sigma^2 \\
\end{aligned}
$$


上面的公式可在某种程度上解释 Bagging 为什么可以减小方差，但是不太能减小偏差。如果 Bagging 的各个学习器之间是独立的，那么根据上式，$\rho=0$，得到 
$$
var\left( \sum_{i=1}^n\frac{x_i}{n}\right) = \frac{\sigma^2}{n}
$$
即，如果学习器之间是独立的，那个 Bagging 在进行学习器结合之后可以将方差减小到原来的 $\frac{1}{n}$。同时还可以看到，结合之后的均值仍然为 $\mu$，没有变化，这说明 Bagging 并不能减小偏差。

但是，在实际运用中，学习器之间的独立性是无法保证的。因此，Bagging 通过 Boostrap 来尽可能保证学习器之间的独立性，同时，RandomForest 还使用了列采样，来进一步增加独立性。

而至于 Boosting 可以减小偏差，是因为 Boosting 是在上一个学习器的基础上进行学习的，主要就是对上一个学习器的偏差进行学习。如 AdaBoost 加大了上一个分类器进行分类错误的样本的权重，而 GBDT 是对于上一个学习器的残差进行学习的，这就类似于一张卷子，第一次做的时候会有一些错误，而之后再做这张卷子的时候对于之前做错的部分加强学习，因此可以达到减小偏差的目的。

但是，由于各个弱学习器之间的强相关的，因此不太能减少方差。

## 13. 随机森林的随机体现在哪些方面

随机森林的随机体现在两个方面。

1. Bootrap 带来的抽样的随机性
2. column sampling 带来的随机性。和传统的决策树不同，Random Forest 的决策树在进行**每个结点**处进行分裂的时候，并不会将所有的特征作为候选特征，而是会进行列采样，采样出 k 个特征作为候选特征，在这 k 个特征中选择最优的特征进行结点分裂。

由于 RandomForest 是 bagging 方法，因此不同基学习器之间的独立性很重要，RandomForest 通过这两种随机性，来保证不同的基学习器之间尽可能独立。

## 14. GBDT 的过程


1. 初始化 $F_0(x)$，$F_0(x)$ 是寻找一个使 loss 最小的常数。即 

$$
F_0(x) = argmin_c \frac{1}{N} \sum_{i=1}^n L(y_i, c)
$$


2. for i = 1 to M
    1. 在学习器 $F_{m-1}\left( x \right)$ 的基础上，学习 $F_m\left( x \right)$ 
以负梯度 $- \frac{\partial L}{\partial F_{m-1} }$  为目标，训练 CART 回归树，只取这个回归树的节点分割方式，而不使用这个回归树在每个叶节点上的输出 $c_{mj}$ ，而 $c_{mj}$ 的计算在下一步进行。
    2. 对于所有的区域 $R_{mj}$ ，计算其上的输出值 $c_{mj}$。，$c_{mj}$ 由下面的式子给出：
    $$
    c_{mj} = \arg \min_c \sum_{x_i \in R_{mj}} L\left(y_i, F_{m-1} (x_i) + c \right) 
    $$ 
    3. 更新 $F_m\left( x \right)  = F_{m-1}\left( x \right) + h_m\left( x \right)$ 
    其中 $h_m\left( x \right) = \sum_{i=1}^{n} I\{ x_i \in R_{mj}\} c_{mj}$  

3. 输出 $F_M\left( x \right)$ 




## GBDT 如何做分类？

GBDT 的基分类器是 CART 回归树，因此只能用来处理连续变量。

通过修改损失函数为 sigmoid or softmax 可以用来进行分类。

对于二分类问题，和回归问题处理基本相似，只不过在负梯度的时候要代入 cross_entropy。同时将输出的值解释为概率。
而对于多分类问题，则需要在每次迭代拟合多棵回归树，并将输出结果进行 softmax。如果是 k 分类问题，那么在每次迭代需要拟合 K 棵回归树。

## XGBoost 对特征的重要性排序原理

XGBoost 对特征的重要性排序有五种方式。分别是 weight、gain、cover、total_gain 和 total_cover。他们的含义分别如下

- weight：某个特征被选择成为分裂特征的次数
- total_gain：某个特征被用来分裂节点时会带来损失下降。这个特征可能不止一次被选择成为分裂特征，total_gain 就是该特征被选择成为分裂特征时带来的损失下降的总和。
- total_cover：某个特征被用来分裂节点时会将多个样本划分到不同的节点上去。这个特征可能不止一次被选择成为分裂特征，total_cover 就是该特征被选择成为分裂特征时分裂的样本的总和。
- gain：total_gain / weight，即分裂一次带来的平均损失下降。
- cover: total_cover / weight，即分裂一个平均的分裂样本数。

## RF 和 GBDT 的异同

相同点：
1. 都是集成学习，都用树作为基学习器

不同点：
1.  随机森林是 bagging 方法，可以并行生成，目的是减小方差。GBDT 是 boosting 方法，可以串行生成，目的是减小偏差。
2. 随机森林对异常值不敏感，而GBDT对异常值非常敏感。因为随机森林会进行列抽样，即某个特征有异常值也有一定概率不抽取到这个特征。而 GBDT 如果某个特征上有异常值，那么这个特征会带来大的偏差，那么下次迭代会对这部分偏差进行重点拟合，因此会使用模型变差。


# References
1. [珍藏版 | 20道XGBoost面试题 - 云+社区 - 腾讯云](https://cloud.tencent.com/developer/article/1500914)
2. [CTR 预测理论（十九）：高维稀疏特征场景中 LR 比 GBDT 效果好的原因_CTR 预测理论（十九）,高维稀疏特征场景,LR_Dby_freedom的博客-CSDN博客](https://blog.csdn.net/Dby_freedom/article/details/98658805?depth_1-utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-1&utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-1)
3. [(2条消息)推荐系统面试题之机器学习（一） -----树模型_jamexfx的博客-CSDN博客_机器学习推荐系统 面试问题](https://blog.csdn.net/jamexfx/article/details/93780308)